In [1]:
# append path info to access api
import os, sys
#from git_root import git_root
#sys.path.append(os.path.abspath(git_root("nmdc_runtime/api/core")))
#sys.path.append(os.path.abspath(git_root("nmdc_runtime/site")))

In [2]:
%load_ext autoreload
%autoreload 2

In [28]:
import json
import copy
import pandas as pd
from toolz.dicttoolz import assoc_in, merge, keymap
from pprint import pprint
from dotenv import dotenv_values
from pymongo import MongoClient

from nmdc_runtime.api.core.metadata import load_changesheet, update_mongo_db, map_id_to_collection, get_collection_for_id
from nmdc_runtime.util import nmdc_jsonschema, nmdc_jsonschema_validate
from linkml_runtime.utils.schemaview import SchemaView

### load mongodb via env info

In [7]:
config = dotenv_values("../../.env.localhost")
config["MONGO_HOST"]

'mongodb://localhost:27018'

In [19]:
client = MongoClient(host=config["MONGO_HOST"], username=config["MONGO_USERNAME"], password=config["MONGO_PASSWORD"])
mongodb = client["nmdc_etl_staging"]

### ensure indexes on mongodb collections

In [20]:
collection_names = {
    name for name, spec in nmdc_jsonschema["properties"].items()
    if name.endswith("_set")
}

# this causes the notebook to hang, so commenting out
# for name in collection_names:
#     mongodb[name].create_index("id", unique=True)

### load change sheet

In [21]:
pd.set_option("display.max_columns", None)
pd.set_option('display.width', 1000)

In [252]:
pd.read_csv("data/changesheet-without-separator3.tsv", sep="\t", dtype="string").fillna('')

,id,action,attribute,value
0,gold:Gs0110138,update,name,soil study
1,,,ecosystem,soil
2,,update,doi,v1
3,v1,,has_raw_value,10.9999/8888
4,,update,principal_investigator,v2
5,v2,,name,fizz buzz
6,v2,,has_raw_value,foo bar
7,,update,description,short description
8,,,websites,http://example.com
9,,,has_credit_associations,v3


In [251]:
sheetDf = load_changesheet('data/changesheet-without-separator3.tsv', mongodb)
sheetDf

,id,action,attribute,value,group_id,group_var,path,collection_name,class_name,prop_depth,prop_range,item_type
0,gold:Gs0110138,update,name,soil study,gold:Gs0110138,,name,test.study_set,Study,1,string,
1,,update,ecosystem,soil,gold:Gs0110138,,ecosystem,test.study_set,Study,1,string,
2,,update,doi,v1,gold:Gs0110138,v1,,test.study_set,Study,,,
3,v1,update,has_raw_value,10.9999/8888,gold:Gs0110138,v1,doi.has_raw_value,test.study_set,Study,2,object:AttributeValue,
4,,update,principal_investigator,v2,gold:Gs0110138,v2,,test.study_set,Study,,,
5,v2,update,name,fizz buzz,gold:Gs0110138,v2,principal_investigator.name,test.study_set,Study,2,object:PersonValue,
6,v2,update,has_raw_value,foo bar,gold:Gs0110138,v2,principal_investigator.has_raw_value,test.study_set,Study,2,object:PersonValue,
7,,update,description,short description,gold:Gs0110138,,description,test.study_set,Study,1,string,
8,,update,websites,http://example.com,gold:Gs0110138,,websites,test.study_set,Study,1,array,string
9,,update,has_credit_associations,v3,gold:Gs0110138,v3,,test.study_set,Study,,,


### update mongodb using change sheet

In [253]:
update_mongo_db(sheetDf, mongodb, print_update=False, print_data=False, print_query=False)

{'n': 9, 'nModified': 1, 'ok': 1.0}